In [ ]:
import pandas as pd;
import matplotlib.pyplot as plt;
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")
import seaborn as sns
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', 150)
pd.set_option('display.max_columns', None)
pd.set_option('float_format', '{:.2f}'.format)

In [ ]:
loan_df = pd.read_csv('loan-Copy1.csv', low_memory=False)

In [ ]:
loan_df.shape

In [ ]:
loan_df.describe()

**We will analyse only closed loans and drop all rows where the loan status is 'Current'.**

In [ ]:
loan_df.loan_status.value_counts()

In [ ]:
loan_df.loan_status = loan_df.loan_status.apply(lambda x : x.strip().upper())

In [ ]:
# Drop loans which are active i.e there loan_status is CURRENT 
loan_df = loan_df[loan_df.loan_status != 'CURRENT']

### Drop columns with all NA values

In [ ]:
loan_df.dropna(axis=1, inplace=True, how='all')


In [ ]:
# Missing percentage
missing_percent = 100*round(loan_df.isnull().sum()/len(loan_df.index),2)

In [ ]:
missing_percent[missing_percent !=0]

In [ ]:
#Dropping further columns with more than 50% missing values like mths_since_last_delinq and mths_since_last_record
loan_df = loan_df.drop(['mths_since_last_record','mths_since_last_delinq'],axis=1)

In [ ]:
loan_df.shape

In [ ]:
# Missing percentage
missing_percent = 100*round(loan_df.isnull().sum()/len(loan_df.index),2)

In [ ]:
missing_percent[missing_percent !=0]

**1. Drop some other columns with ignorable info like desc**

**2. Columns with unique identifier member_id','url' and'id'** 

**3. Remove rows where emp_title, pub_rec_bankruptcies,  emp_length have null values as the percentage of it is very low**

In [ ]:
loan_df = loan_df.drop(['desc','url','member_id','id'],axis=1)

In [ ]:
loan_df.dropna(inplace=True, how='any')

In [ ]:
missing_percent = 100*round(loan_df.isnull().sum()/len(loan_df.index),2)

In [ ]:
missing_percent[missing_percent !=0]

## **Now we don't have any missing values**

In [ ]:
loan_df.info()

In [ ]:
loan_df.collections_12_mths_ex_med.value_counts(dropna=False)

In [ ]:
loan_df.chargeoff_within_12_mths.value_counts(dropna=False)

In [ ]:
loan_df.policy_code.value_counts(dropna=False)

In [ ]:
loan_df.tax_liens.value_counts(dropna=False)

In [ ]:
loan_df.application_type.value_counts(dropna=False)

In [ ]:
loan_df.acc_now_delinq.value_counts(dropna=False)

In [ ]:
loan_df.pymnt_plan.value_counts(dropna=False)

In [ ]:
loan_df.initial_list_status.value_counts(dropna=False)

In [ ]:
loan_df.zip_code.value_counts(dropna=False)

In [ ]:
loan_df.delinq_amnt.value_counts(dropna=False)

**1. Since the values of**

    pymnt_plan, 
    acc_now_delinq, 
    application_type, 
    tax_liens, 
    policy_code,
    collections_12_mths_ex_med, delinq_amnt,  
    initial_list_status and chargeoff_within_12_mths 
    
   **are same for all rows, we will drop these columns**

**2. The zip_code doesn't add value and hence will be dropped too.**


In [ ]:
loan_df = loan_df.drop(['zip_code','pymnt_plan', 'acc_now_delinq','delinq_amnt','application_type','tax_liens','policy_code','chargeoff_within_12_mths','collections_12_mths_ex_med','initial_list_status'],axis=1)

In [ ]:
loan_df[loan_df['loan_amnt'] != loan_df['funded_amnt']]

**We have 1643 rows where the funded_amt is less than the loan_amt. This difference can be a insight to be analysed ?**

But the numbers are very less and hence we can drop the funded_amt too

**funded_amnt_inv is the total amount committed by the investor after decision making of the loan application and  hence can be dropped.**

**Title is free text field containing need for loan provided by the customer. This has been already categorized neatly via the purpose column. So we can drop the title column.**

In [ ]:
loan_df = loan_df.drop(['funded_amnt','funded_amnt_inv','title'],axis=1)

In [ ]:
loan_df.shape

## Standardize data and fix missing values

Imputing Missing Values

- For numerical variables use mean and median
- For categorical variables use mode


### Remove % from interest rate , revol_util column and convert to float

In [ ]:
loan_df.int_rate = loan_df.int_rate.apply(lambda x : float(x.replace('%','')))


In [ ]:
loan_df.revol_util = loan_df.revol_util.apply(lambda x : float(x.replace('%','')))

In [ ]:
loan_df.term.value_counts()

### Strip whitespaces from term and remove 'months'

In [ ]:
loan_df.term = loan_df.term.apply(lambda x : x.strip().replace('months',''))

In [ ]:
loan_df.emp_length.value_counts(dropna=False, normalize=True)

- Convert all < 1 year to 0 
- Convert all 10+ year to 10
- Remove year|years

In [ ]:
#loan_df.emp_length = loan_df.emp_length.apply(lambda x : int(x.strip().replace('years','').replace('year','').replace('+','').replace('< 1','0')))

#loan_df.emp_length = loan_df.emp_length.apply(lambda x : int(x.strip().replace('years','').replace('year','').replace('+','').replace('< 1','0')))

In [ ]:
#loan_df.rename(columns={"emp_length": "emp_length_year"}, inplace=True)

In [ ]:
#loan_df.emp_length_year.value_counts()

### Convert the emp_title to lower case and also strip off any non alphabet characters. 


In [ ]:
loan_df.emp_title = loan_df.emp_title.str.lower().str.replace('[^a-zA-Z]', '')

In [ ]:
loan_df.emp_title.value_counts(dropna=False)

In [ ]:
loan_df.home_ownership.value_counts(dropna=False)

### Change NONE value to mode value of the column as the only valid values are RENT, OWN, MORTGAGE, OTHER

In [ ]:
loan_df.home_ownership.mode()[0]

In [ ]:
loan_df.loc[loan_df.home_ownership == 'NONE',['home_ownership']] = 'RENT'

In [ ]:
loan_df.home_ownership.value_counts(dropna=False)

In [ ]:
loan_df.dtypes[loan_df.dtypes != object].index

In [ ]:
loan_df.dtypes[loan_df.dtypes == object].index

In [ ]:
len(loan_df.dtypes[loan_df.dtypes != object].index)

In [ ]:
len(loan_df.dtypes[loan_df.dtypes == object].index)

Round of to two decimal places

In [ ]:
loan_df.total_pymnt = loan_df.total_pymnt.round(2)
loan_df.total_pymnt_inv = loan_df.total_pymnt_inv.round(2)
loan_df.total_rec_int = loan_df.total_rec_int.round(2)
loan_df.recoveries = loan_df.total_rec_int.round(2)
loan_df.dti = loan_df.dti.round(2)

In [ ]:
loan_df.sort_index(axis = 1, inplace=True)

In [ ]:
loan_df.reset_index(inplace=True,drop=True)

In [ ]:
loan_df.to_csv('filtered.csv',index=False)

### Split the issue_d into issue_month and issue_year

**Derived Columns**

Issue month and year

Loan Ratio


In [ ]:
loan_df['issue_month'], loan_df['issue_year'] = loan_df.issue_d.str.split('-').str

In [ ]:
loan_df['loan_ratio'] = loan_df['loan_amnt']/loan_df['annual_inc']

In [ ]:
loan_df[['loan_ratio','issue_month','issue_year','loan_amnt','annual_inc']].head()

In [ ]:
loan_df.annual_inc.describe(percentiles=[0.1, 0.25, 0.5, 0.75, 0.985])

In [ ]:
plt.figure(figsize=(15, 3))
sns.boxplot(x=loan_df['annual_inc'], data=loan_df)
plt.show()

**We see outliers for annual_inc which can be removed from analysis. 98.5% is 2K Annual income. 
We will drop all rows with annual_inc above this**

In [ ]:
loan_df= loan_df[loan_df.annual_inc <= loan_df["annual_inc"].quantile(.985)]

In [ ]:
plt.figure(figsize=(15, 3))
sns.boxplot(x=loan_df['annual_inc'], data=loan_df)
plt.show()

In [ ]:
loan_df['annual_inc_range'] = pd.cut(loan_df.annual_inc,bins=[0, 20000, 40000, 60000, 80000, 100000, 200000],labels= ['< 20k', '20k - 40k', '40k - 60k', '60k - 80k', '80k - 1L','> 1L'])

In [ ]:
loan_df.annual_inc_range.value_counts()

In [ ]:
loan_df.int_rate.describe(percentiles=[0.1, 0.25, 0.5, 0.75, 0.985])

In [ ]:
plt.figure(figsize=(15, 3))
sns.boxplot(x=loan_df['int_rate'], data=loan_df)
plt.show()

In [ ]:
loan_df['int_rate_range'] = pd.cut(loan_df.int_rate,bins=[0, 6, 8, 12, 15, 25],labels= ['0-6', '6-8', '8-12', '12-15','>15'])

In [ ]:
loan_df.int_rate_range.value_counts()

In [ ]:
loan_df.info()

In [ ]:
loan_df.dtypes[loan_df.dtypes == object].index

## Analysis

**addr_state**

In [ ]:
plt.figure(figsize=(30,16))
sns.countplot(x='addr_state', hue='loan_status', data=loan_df)
plt.show()

 Observation 
 
 **highest shares of loans are from CA FL NY Texas**
 
 It seems the number of loans paid and charged off is having positive correlation


**annual_inc**

In [ ]:
loan_df.annual_inc.describe()

In [ ]:
plt.figure(figsize=(20, 20))
plt.subplot(4,3,1)
loan_df.annual_inc_range.value_counts().plot.pie(autopct='%.2f%%')
plt.title('Annual income range')

plt.subplot(4,3,2)
loan_df.loan_status.value_counts().plot.pie(autopct='%.2f%%')
plt.title('Loan Status')

plt.subplot(4,3,3)
loan_df.term.value_counts().plot.pie(autopct='%.2f%%')
plt.title('Term of Loan')

plt.subplot(4,3,4)
loan_df.emp_length.value_counts().plot.pie(autopct='%.2f%%')
plt.title('Employment Length')


plt.subplot(4,3,5)
loan_df.int_rate_range.value_counts().plot.pie(autopct='%.2f%%')
plt.title('Interest Rate range')

plt.subplot(4,3,6)
loan_df.home_ownership.value_counts().plot.pie(autopct='%.2f%%')
plt.title('Home Ownership Distribution')



plt.subplot(4,3,(7,8))
loan_df.addr_state.value_counts().plot(kind='bar')
plt.title('Address State Distribution')


plt.subplot(4,3,9)
loan_df.verification_status.value_counts().plot.pie(autopct='%.2f%%')
plt.title('Verification Status Distribution')

plt.subplot(4,3,10)
loan_df.purpose.value_counts().plot.bar()
plt.title('Purpose Distribution')

plt.subplot(4,3,11)
loan_df.grade.value_counts().plot.bar()
plt.title('Loan grade Distribution')

#plt.subplot(4,3,(11,12))
#loan_df.sub_grade.value_counts().plot.bar()
#plt.title('Loan grade Distribution')

plt.show()



In [ ]:
loan_df.dtypes[loan_df.dtypes != object].index

In [ ]:
plt.figure(figsize=(20, 20))
plt.subplot(4,3,1)
sns.boxplot(y='loan_amnt', x='loan_status', data=loan_df )   
plt.subplot(4,3,2)
sns.boxplot(y='int_rate', x='loan_status', data=loan_df ) 
plt.subplot(4,3,3)
sns.boxplot(y='annual_inc', x='loan_status', data=loan_df ) 
plt.subplot(4,3,4)
sns.boxplot(y='dti', x='loan_status', data=loan_df ) 
plt.subplot(4,3,5)
sns.boxplot(y='last_pymnt_amnt', x='loan_status', data=loan_df ) 
plt.show()

In [ ]:
loan_df.out_prncp.value_counts()

In [ ]:
loan_df.out_prncp_inv.value_counts()

In [ ]:
plt.figure(figsize=(20, 20))
sns.heatmap(loan_df.corr(),annot=True,linewidths=.1)
plt.show()